In [1]:
import re
import unicodedata
import string
import random
import nltk
from nltk.probability import ConditionalFreqDist
import ipywidgets as widgets
# from ipython.display import display

In [2]:
def filter(text):
    # normalize text
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    # replace html chars with ' '
    text = re.sub('<.*?>', ' ', text)
    # remove punctuation
    text = text.translate(str.maketrans(' ', ' ', string.punctuation))
    # only alphabets and numerics
    text = re.sub('[^a-zA-Z]', ' ', text)
    # replace newline with space
    text = re.sub("\n", " ", text)
    # lower case
    text = text.lower()
    # split and join the words
    text = ' '.join(text.split())

    return text


In [3]:
def tokenize(text):
    tokenized = nltk.word_tokenize(text)
    lematized = nltk.stem.WordNetLemmatizer()
    final = []
    for words in tokenized:
        final.append(lematized.lemmatize(words))
    return final


In [4]:
def n_gram_model(text):
    trigram = list(nltk.ngrams(text, 3, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))

    cfd = ConditionalFreqDist()
    for w1, w2, w3 in trigram:
        cfd[(w1, w2)][w3] += 1

    for w1_w2 in cfd:
        total_count = float(sum(cfd[w1_w2].values()))
        for w3 in cfd[w1_w2]:
            cfd[w1_w2][w3] /= total_count

    return cfd

In [5]:
def predict(model, text_input):
    text_input = filter(text_input)
    text_input = text_input.split()

    w1 = len(text_input) - 2
    w2 = len(text_input)
    prev_words = text_input[w1:w2]

    prediction = sorted(dict(model[prev_words[0], prev_words[1]]), key=lambda x: dict(model[prev_words[0], prev_words[1]])[x], reverse=True)
    word = []
    weight = []
    for key, prob in dict(model[prev_words[0], prev_words[1]]).items():
        word.append(key)
        weight.append(prob)
    next_word = random.choices(word, weights=weight, k=1)
    text_input.append(next_word[0])
    print(' '.join(text_input))
    print(prediction[:10])


In [6]:
with open('/Users/akheels/Untitled Folder/data2.txt','r') as file:
    text = file.read()

    text_corpus = filter(text)
    text_corpus = tokenize(text_corpus)
    model = n_gram_model(text_corpus)

In [7]:
text = widgets.Text()
btn = widgets.Button(description = 'Predict')
display(text, btn)

def prediction(text_object):
    predict(model, text.value)

btn.on_click(prediction)

Text(value='')

Button(description='Predict', style=ButtonStyle())

what is the
['the', 'your', 'a', 'this', 'it', 'something', 'going', 'wrong', 'one', 'that']
